Transfer learning is when a model developed for one task is reused to work on a second task. 
Fine tuning is one approach to transfer learning.

BERT (Bidirectional Encoder Representations from Transformers) is a big neural network architecture, with a huge number of parameters, that can range from 100 million to over 300 million. So, training a BERT model from scratch on a small dataset would result in overfitting.

So, it is better to use a pre-trained BERT model that was trained on a huge dataset, as a starting point. We can then further train the model on our relatively smaller dataset and this process is known as model fine-tuning.
“BERT stands for Bidirectional Encoder Representations from Transformers. It is designed to pre-train deep bidirectional representations from unlabeled text by jointly conditioning on both left and right context. As a result, the pre-trained BERT model can be fine-tuned with just one additional output layer to create state-of-the-art models for a wide range of NLP tasks.”
Different Fine-Tuning Techniques
Train the entire architecture – We can further train the entire pre-trained model on our dataset and feed the output to a softmax layer. In this case, the error is back-propagated through the entire architecture and the pre-trained weights of the model are updated based on the new dataset.
Train some layers while freezing others – Another way to use a pre-trained model is to train it partially. What we can do is keep the weights of initial layers of the model frozen while we retrain only the higher layers. We can try and test as to how many layers to be frozen and how many to be trained.
Freeze the entire architecture – We can even freeze all the layers of the model and attach a few neural network layers of our own and train this new model. Note that the weights of only the attached layers will be updated during model training.


# Install Needed libraries

In [ ]:
!pip install tensorflow_hub
!pip install bert-for-tf2
!pip install tensorflow
!pip install sentencepiece
!pip install transformers
!pip install torchvision 

# Import Needed Libraries

In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
import tensorflow_hub as hub
import bert
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
from transformers import DistilBertModel,DistilBertTokenizer
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tensorflow.keras.layers import Dense, Input
from sklearn.metrics import classification_report
from tensorflow.keras import backend as K
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings("ignore")
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# load Bert tokenizer

In [ ]:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
from transformers import AutoModel, AutoTokenizer

# Load pretrained model/tokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/distilbert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/distilbert-base-turkish-cased")

# Load Dataset

In [ ]:
dataset = pd.read_csv("/kaggle/input/turkish-sentiment-analysis-data-beyazperdecom/train.csv")
test = pd.read_csv("/kaggle/input/turkish-sentiment-analysis-data-beyazperdecom/test.csv")
dataset.info()

In [ ]:
#number of labels
len(set(dataset["Label"].values))

# Helper Functions

In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)

def preprocess_text(sen):
    # Removing html tags
    sentence = remove_tags(sen)

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing newlines
    sentence = re.sub(r'\n', ' ', sentence)
    
    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence
dataset["comment"] = dataset["comment"].apply(lambda x : preprocess_text(str(x)))
dataset["comment"] = dataset["comment"].apply(lambda x : ' '.join([w.strip() for w in x.split() if w!=' ']))

test["comment"] = test["comment"].apply(lambda x : preprocess_text(str(x)))
test["comment"] = test["comment"].apply(lambda x : ' '.join([w.strip() for w in x.split() if w!=' ']))

In [ ]:
def summarize_model(history):
    pyplot.subplot(211)
    pyplot.title('Loss')
    pyplot.plot(history.history['loss'], label='train')
    pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    # plot accuracy during training
    pyplot.subplot(212)
    pyplot.title('Accuracy')
    pyplot.plot(history.history['accuracy'], label='train')
    pyplot.plot(history.history['val_accuracy'], label='test')
    pyplot.legend()
    pyplot.show()

# Data Statistics

In [ ]:
train_x,eval_x,train_y,eval_y = train_test_split(dataset["comment"],dataset["Label"],test_size=0.35)

In [ ]:
print("length of train set:",len(train_x))
print("length of validate set:",len(eval_x))
print("length of test set:",len(test))

# Prepare Data for Bert 

In [ ]:
tokenized_tr=tokenizer.batch_encode_plus(train_x.tolist(),max_length =100,return_attention_mask=True,padding=True)

In [ ]:
tokenized_vl=tokenizer.batch_encode_plus(eval_x.tolist(),max_length =100,return_attention_mask=True,padding=True)

In [ ]:
tokenized_ts=tokenizer.batch_encode_plus(test["comment"].tolist(),max_length =100,return_attention_mask=True,padding=True)

# Build Model

In [ ]:
def get_features(padded,attention):
    input_ids = torch.tensor(padded,dtype=torch.int64)  
    attention_mask = torch.tensor(attention,dtype=torch.int64)
    with torch.no_grad():
        last_hidden_states = model(input_ids,attention_mask)
    features = last_hidden_states[0][:,0,:].numpy()
    return features

In [ ]:
features_tr = get_features(tokenized_tr['input_ids'],tokenized_tr['attention_mask'])

In [ ]:
features_vl = get_features(tokenized_vl['input_ids'],tokenized_vl['attention_mask'])

In [ ]:
features_ts = get_features(tokenized_ts['input_ids'],tokenized_ts['attention_mask'])

**Traditional Machine Learning Algorithms**

In [ ]:
#logistic regression
from sklearn.linear_model import LogisticRegression
lr_clf = LogisticRegression()
lr_clf.fit(features_tr, train_y)
print('train score:',lr_clf.score(features_tr, train_y))
print('validation score:',lr_clf.score(features_vl, eval_y))
print('test score:',lr_clf.score(features_ts, test['Label']))

In [ ]:
#naive bayes
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(features_tr, train_y)
print('train score:',gnb.score(features_tr, train_y))
print('validation score:',gnb.score(features_vl, eval_y))
print('test score:',gnb.score(features_ts, test['Label']))

**Deep Learning Models**

In [ ]:
from tensorflow.keras.layers import Input,concatenate,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import tensorflow as tf 

input1 = Input(shape=(features_tr.shape[1],))
dense1 = Dense(128,activation='relu')(input1)
dense2 = Dense(1,activation='sigmoid')(dense1)
tfmodel = Model(inputs=input1,outputs=dense2)

In [ ]:
tf.keras.utils.plot_model(tfmodel, show_shapes=True, dpi=48)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy (from_logits=False)
optimizer = keras.optimizers.Adam(lr=1e-3,decay=1e-3/32)
tfmodel.compile(optimizer=optimizer, loss=[loss, loss],metrics=["accuracy"])
checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy')
fine_history = tfmodel.fit(features_tr, train_y, validation_data=(features_vl,eval_y),
                          epochs=3, callbacks=[checkpoint, earlystopping],batch_size=32,verbose=1)

In [ ]:
summarize_model(fine_history)

**Evaluation**

In [ ]:
#training set
y_pred=tfmodel.predict(features_tr)
y_pred = y_pred.round()
print(classification_report(train_y,y_pred))

In [ ]:
#validation set
y_pred=tfmodel.predict(features_vl)
y_pred = y_pred.round()
print(classification_report(eval_y,y_pred))

In [ ]:
#testing set
y_pred=tfmodel.predict(features_ts)
y_pred = y_pred.round()
print(classification_report(test['Label'],y_pred))